In [ ]:
!pip install -q transformers==4.51.3 accelerate==1.7.0 datasets==2.20.0 sqlalchemy==2.0.30 sqlite-utils==3.38 evaluate==0.4.3

^C


In [ ]:
!pip list | grep -E 'transformers|accelerate|datasets|sqlalchemy|sqlite-utils|moz-sql-parser|evaluate|torch|torchvision'

accelerate                         1.7.0
datasets                           2.20.0
evaluate                           0.4.3
sentence-transformers              3.4.1
sqlite-utils                       3.38
tensorflow-datasets                4.9.7
torch                              2.2.2
torchaudio                         2.5.1+cu124
torchsummary                       1.5.1
torchvision                        0.17.2
transformers                       4.51.3
vega-datasets                      0.9.0


In [ ]:
%%bash
python - <<'PY'
import random, numpy as np, torch, os
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
print("Seeds fixed to", SEED)
PY


Seeds fixed to 42


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login

login()

In [ ]:
from datasets import load_dataset

spider_val = load_dataset("spider")["validation"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
spider_val

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 1034
})

In [ ]:
# "meta-llama/Llama-3.1-8B"
# "deepseek-ai/deepseek-llm-7b-base"
# deepseek-ai/deepseek-r1-9b-base
# deepseek-ai/deepseek-coder-6.7b-base
# "mistral"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "deepseek-ai/deepseek-coder-6.7b-base"  # use preferred model
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-06)
    (rotary_e

In [ ]:
!pip install -q kagglehub            # tiny, pure-python package
import kagglehub, os, json, zipfile
from pathlib import Path

# Either:
#   kagglehub.login()                              # will prompt
#   – or –
os.environ["KAGGLE_USERNAME"] = ""        # <- set once per session
os.environ["KAGGLE_KEY"]     = ""


In [ ]:
import kagglehub, zipfile
from pathlib import Path

root = Path( kagglehub.dataset_download(
           "jeromeblanchet/yale-universitys-spider-10-nlp-dataset") )

# 1️⃣  Look for a zip first (old mirror versions)
zip_files = list(root.rglob("spider.zip"))
if zip_files:
    print("Found spider.zip – extracting…")
    with zipfile.ZipFile(zip_files[0], "r") as zf:
        zf.extractall("spider")          # creates ./spider/database/…
    SPIDER_DIR = Path("spider")

# 2️⃣  Otherwise assume the folder is already there (current mirror)
else:
    print("No spider.zip – using unpacked folder")
    # the mirror root itself is usually “…/spider/”, but we search just in case
    try:
        SPIDER_DIR = next(root.rglob("spider/database")).parent
    except StopIteration as e:
        raise FileNotFoundError(
            "Could not locate Spider data inside the mirror download") from e

DB_ROOT = SPIDER_DIR / "database"
print("Spider databases live at:", DB_ROOT.resolve())


No spider.zip – using unpacked folder
Spider databases live at: /kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database


In [ ]:
DB_ROOT = Path("/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database")

In [ ]:
# reflect every .sqlite file to create a plain-text schema description


import sqlalchemy as sa

_schema_cache: dict[str, str] = {}            # {db_id: "…schema string…"}

def get_schema_string(db_id: str) -> str:
    """Return a compact textual schema for the given Spider database."""
    if db_id in _schema_cache:
        return _schema_cache[db_id]

    db_file = DB_ROOT / db_id / f"{db_id}.sqlite"
    engine  = sa.create_engine(f"sqlite:///{db_file}")
    insp    = sa.inspect(engine)

    parts   = []
    for tbl in sorted(insp.get_table_names()):
        cols = [c["name"] for c in insp.get_columns(tbl)]
        parts.append(f"{tbl}({', '.join(cols)})")

    schema_str = ", ".join(parts)
    _schema_cache[db_id] = schema_str
    return schema_str

In [ ]:
def build_prompt(nl_question: str, schema: str) -> str:
    return (
        "### Instruction:\n"
        "You are an expert SQL developer. Given a database schema and a natural-language question,\n"
        "write ONE syntactically correct SQL query that answers the question.\n\n"

        f"### Database Schema:\n{schema}\n\n"

        f"### Question:\n{nl_question}\n\n"

        "### SQL:\n")


In [ ]:
# import re

# def generate_sql(nl_question: str, schema: str) -> str:
#     prompt     = build_prompt(nl_question, schema)
#     inputs     = tokenizer(prompt, return_tensors="pt").to(model.device)
#     input_len  = inputs["input_ids"].shape[1]

#     model.config.pad_token_id = tokenizer.eos_token_id
#     out = model.generate(
#         **inputs,
#         max_new_tokens=128,
#         num_beams=5,
#         early_stopping=True,
#         do_sample=False,
#     )
#     gen_text = tokenizer.decode(out[0, input_len:], skip_special_tokens=True)
#     sql = re.split(r"(###|\n\s*\n|```)", gen_text, maxsplit=1)[0]
#     sql = sql.split(";", 1)[0]
#     sql = sql.split(";")[0].replace("\n", " ").strip()
#     return sql.strip()



In [ ]:
# def generate_sql(nl_question: str, schema: str) -> str:
#     prompt     = build_prompt(nl_question, schema)
#     inputs     = tokenizer(prompt, return_tensors="pt").to(model.device)
#     input_len  = inputs["input_ids"].shape[1]

#     model.config.pad_token_id = tokenizer.eos_token_id

#     out        = model.generate(
#         **inputs,
#         max_new_tokens=128,
#         num_beams=5,
#         early_stopping=True,
#         do_sample=False,
#     )

#     gen_tokens = out[0, input_len:]
#     sql        = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

#     # trim markdown fences etc.
#     #sql = sql.split("```")[-1].strip()
#     #sql = sql.split(";")[0].replace("\n", " ").strip()
#     #return sql

#     # remove prefix for llama3.1
#     prefix = "```sql "
#     if sql.startswith(prefix):
#       sql = sql[len(prefix):]
#     else:
#       sql = sql


#     return sql


In [ ]:
# ──────────────────────────────────────────────────────────────────────────
# 0 ▪ global seed (reproducible shuffles, torch, numpy, random, etc.)
# ──────────────────────────────────────────────────────────────────────────
import random, numpy as np, torch, os
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# ──────────────────────────────────────────────────────────────────────────
# 1 ▪ zero-shot prompt builder (no demos)
# ──────────────────────────────────────────────────────────────────────────
def build_prompt_zero(nl_question: str, target_schema: str) -> str:
    return (
        "### Instruction:\n"
        "You are an expert SQL developer. Given the database schema and the "
        "question, return ONE valid SQL statement — output ONLY the SQL.\n\n"
        f"### Database Schema:\n{target_schema}\n\n"
        f"### Question:\n{nl_question}\n\n"
        "### SQL:\n"
    )

# ──────────────────────────────────────────────────────────────────────────
# 2 ▪ random-5-shot builder
# ──────────────────────────────────────────────────────────────────────────
NUM_SHOTS = 5
spider_train = load_dataset("spider")["train"]
DEMO_SET = [
    {**ex, "schema_str": get_schema_string(ex["db_id"])}
    for ex in spider_train.shuffle(seed=SEED).select(range(NUM_SHOTS))
]

def build_prompt_random5(nl_question: str, target_schema: str) -> str:
    parts = ["### Instruction:\nReturn ONE SQL query only, based on the examples provided.\n"]
    for i, ex in enumerate(DEMO_SET, 1):
        parts += [
            f"### Example {i} Schema:\n{ex['schema_str']}",
            f"### Example {i} Question:\n{ex['question']}",
            f"### Example {i} SQL:\n{ex['query'].strip()}",
        ]
    parts += [
        f"### Database Schema:\n{target_schema}",
        f"### Question:\n{nl_question}",
        "### SQL:\n",
    ]
    return "\n".join(parts)

# ──────────────────────────────────────────────────────────────────────────
# 3 ▪ retrieved-5-shot builder (SBERT nearest neighbours)
# ──────────────────────────────────────────────────────────────────────────
from sentence_transformers import SentenceTransformer, util
embedder   = SentenceTransformer("all-MiniLM-L6-v2")
train_emb  = embedder.encode(spider_train["question"], convert_to_tensor=True)

def get_k_shots(nl_question: str, k: int = 5):
    q_emb  = embedder.encode(nl_question, convert_to_tensor=True)
    hits   = util.semantic_search(q_emb, train_emb, top_k=k)[0]
    demos  = []
    for h in hits:
        ex = spider_train[int(h["corpus_id"])]
        demos.append({
            "question"   : ex["question"],
            "query"      : ex["query"],
            "schema_str" : get_schema_string(ex["db_id"]),
        })
    return demos

def build_prompt_retrieved5(nl_question: str, target_schema: str) -> str:
    demos = get_k_shots(nl_question, 5)
    parts = ["### Instruction:\nReturn ONE SQL query only, based on the examples provided.\n"]
    for i, ex in enumerate(demos, 1):
        parts += [
            f"### Example {i} Schema:\n{ex['schema_str']}",
            f"### Example {i} Question:\n{ex['question']}",
            f"### Example {i} SQL:\n{ex['query'].strip()}",
            "### End\n",
        ]
    parts += [
        f"### Database Schema:\n{target_schema}",
        f"### Question:\n{nl_question}",
        "### SQL:\n",
    ]
    return "\n".join(parts)

# ──────────────────────────────────────────────────────────────────────────
# 4 ▪ shared generate_sql that accepts a *builder* argument
# ──────────────────────────────────────────────────────────────────────────
import re

def make_generator(builder_fn):
    def _gen(nl_question: str, schema: str) -> str:
        prompt = builder_fn(nl_question, schema)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        input_len = inputs["input_ids"].shape[1]

        model.config.pad_token_id = tokenizer.eos_token_id
        out = model.generate(
            **inputs,
            max_new_tokens=128,
            num_beams=3,
            early_stopping=True,
            do_sample=False,
        )
        gen_text = tokenizer.decode(out[0, input_len:], skip_special_tokens=True)
        sql = re.split(r"(###|\n\s*\n|```)", gen_text, 1)[0]
        sql = sql.split(";")[0].replace("\n", " ").strip()
        return sql.split(";", 1)[0].strip()
    return _gen

# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 0          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

In [ ]:
# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 2          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

NameError: name 'build_prompt_zero' is not defined

In [ ]:
example      = spider_val[325]
db_id        = example["db_id"]
schema_str   = get_schema_string(db_id)
predicted_sql= generate_sql(example["question"], schema_str)
print(example["query"])
print(predicted_sql)

SELECT count(*) FROM Templates WHERE template_type_code  =  "CV"
SELECT count(*) FROM Templates WHERE template_type_code  =  "CV"


In [ ]:
import re
_ws   = re.compile(r"\s+")
_comm = re.compile(r"\s*,\s*")          # blank(s) ↔ comma ↔ blank(s)

def exact_match(pred: str, gold: str) -> bool:
    def norm(s: str) -> str:
        s = s.strip().lower()
        s = _ws.sub(" ", s)             # collapse runs of whitespace
        s = _comm.sub(", ", s)          # canonical “comma␣”
        return s
    return norm(pred) == norm(gold)


In [ ]:
from sqlglot import parse_one, expressions

def flatten_ast(node):
    """
    Recursively collect all node‐type names and literal values as lowercase strings.
    """
    out = set()

    def walk(n):
        # record the AST node type
        out.add(type(n).__name__.lower())

        # record any literal (e.g. identifiers, strings, numbers)
        if hasattr(n, "this") and isinstance(n.this, (str, int, float)):
            out.add(str(n.this).lower())

        # recurse into child expressions
        for arg in n.args.values():
            if isinstance(arg, list):
                for child in arg:
                    if isinstance(child, expressions.Expression):
                        walk(child)
            elif isinstance(arg, expressions.Expression):
                walk(arg)

    walk(node)
    return out

def component_match(pred_sql, gold_sql):
    try:
        pred_ast = parse_one(pred_sql)
        gold_ast = parse_one(gold_sql)
    except Exception:
        return 0.0

    pred_set = flatten_ast(pred_ast)
    gold_set = flatten_ast(gold_ast)
    if not gold_set:
        return 0.0
    return len(pred_set & gold_set) / len(gold_set)


In [ ]:
import sqlite3, pandas as pd, numpy as np
from tqdm import tqdm

def run_query(sql: str, db_path: Path):
    """Return query result as a sorted list of tuples (order-independent)."""
    try:
        with sqlite3.connect(db_path) as conn:
            df = pd.read_sql_query(sql, conn)
        # sort rows + cols for order-invariant comparison
        return tuple(map(tuple, df.sort_index(axis=1).sort_values(list(df.columns)).to_numpy()))
    except Exception as e:
        # any failure counts as wrong
        return f"ERROR-{e}"

def execution_accuracy(dataset):
    """Compute Spider-style Execution Accuracy on a HF split (e.g. validation)."""
    correct = 0
    for ex in tqdm(dataset, desc="Evaluating"):
        db_id   = ex["db_id"]
        schema  = get_schema_string(db_id)
        pred_sql= generate_sql(ex["question"], schema)

        db_file = DB_ROOT / db_id / f"{db_id}.sqlite"
        gold    = run_query(ex["query"],     db_file)
        pred    = run_query(pred_sql,        db_file)

        if gold == pred:
            correct += 1

    return correct / len(dataset)

In [ ]:
spider_val

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 1034
})

In [ ]:
p = 791
print(spider_val[p]['query'])
print(generate_sql(spider_val[p]['question'], get_schema_string(spider_val[p]['db_id'])))

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


SELECT Name ,  population ,  HeadOfState FROM country ORDER BY SurfaceArea DESC LIMIT 1
SELECT Name, Population, HeadOfState FROM country WHERE SurfaceArea = (SELECT MAX(SurfaceArea) FROM country)


In [ ]:
exact_match(generate_sql(spider_val[p]['question'], get_schema_string(spider_val[p]['db_id'])), spider_val[p]['query'])

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


False

In [ ]:
component_match(generate_sql(spider_val[p]['question'], get_schema_string(spider_val[p]['db_id'])), spider_val[p]['query'])

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


0.6666666666666666

In [ ]:
spider_val

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 1034
})

In [ ]:
spider_val.shuffle(seed=42).select(range(100))

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 100
})

SyntaxError: invalid syntax (<ipython-input-91-606067e16966>, line 112)

# Simple Random Sampling of 20% of the validation set.



# zero shot

In [ ]:
# Evaluation on a fixed 20% Spider dev sample
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging
import random

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')

# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 0          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

# 1) Size of a 20 % slice (Spider dev has 1 034 samples)
num_samples = int(0.20 * len(spider_val))      # → 206

sample_val  = spider_val.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_val, desc="Evaluating"):
    db_id   = ex["db_id"]
    schema  = get_schema_string(db_id)

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question"], schema)
    times.append(time.perf_counter() - t0)

    gold_sql= ex["query"]

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    db_path = DB_ROOT / db_id / f"{db_id}.sqlite"
    gold_res= run_query(gold_sql,  db_path)
    pred_res= run_query(pred_sql, db_path)
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match        :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")


Evaluating: 100%|██████████| 206/206 [06:10<00:00,  1.80s/it]



Exact Match        : 0.18932038834951456
Component Match    : 0.8402611461047015
Execution Accuracy : 0.616504854368932
Avg. Latency  (s)  : 1.7106464342766927
95% Latency  (s)   : 4.917845079749895
GPU Mem Peak       : 15.023514624 GB


# Few-shot (Random Sampling)

In [ ]:
# Evaluation on a fixed 20% Spider dev sample
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging
import random

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')

# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 1          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

# 1) Size of a 20 % slice (Spider dev has 1 034 samples)
num_samples = int(0.20 * len(spider_val))      # → 206

sample_val  = spider_val.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_val, desc="Evaluating"):
    db_id   = ex["db_id"]
    schema  = get_schema_string(db_id)

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question"], schema)
    times.append(time.perf_counter() - t0)

    gold_sql= ex["query"]

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    db_path = DB_ROOT / db_id / f"{db_id}.sqlite"
    gold_res= run_query(gold_sql,  db_path)
    pred_res= run_query(pred_sql, db_path)
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match    :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")


Evaluating: 100%|██████████| 206/206 [21:12<00:00,  6.18s/it]



Exact Match        : 0.1553398058252427
Component Match    : 0.9037864636666968
Execution Accuracy : 0.5728155339805825
Avg. Latency  (s)  : 6.0849226715825315
95% Latency  (s)   : 6.54172776099972
GPU Mem Peak       : 19.78111744 GB


# Few-shot (all-MiniLM)

In [ ]:
# Evaluation on a fixed 20% Spider dev sample
import time, torch, numpy as np
from tqdm import tqdm
import warnings
from transformers import logging
import random

random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings('ignore')

# choose the prompt mode here ─────────────────────────────────────────────
PROMPT_MODE = 2          # 0 = zero-shot, 1 = random-5, 2 = retrieved-5
builder = [build_prompt_zero,
           build_prompt_random5,
           build_prompt_retrieved5][PROMPT_MODE]
generate_sql = make_generator(builder)

# 1) Size of a 20 % slice (Spider dev has 1 034 samples)
num_samples = int(0.20 * len(spider_val))      # → 206

sample_val  = spider_val.shuffle(seed=42).select(range(num_samples))


# 2) containers
em_scores, cm_scores, ex_scores, times = [], [], [], []

# 3) start fresh CUDA-peak tracking
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 4) main loop
for ex in tqdm(sample_val, desc="Evaluating"):
    db_id   = ex["db_id"]
    schema  = get_schema_string(db_id)

    t0      = time.perf_counter()
    pred_sql= generate_sql(ex["question"], schema)
    times.append(time.perf_counter() - t0)

    gold_sql= ex["query"]

    # exact + component match
    em_scores.append( float(exact_match(pred_sql, gold_sql)) )
    cm_scores.append( component_match(pred_sql, gold_sql) )

    # execution accuracy
    db_path = DB_ROOT / db_id / f"{db_id}.sqlite"
    gold_res= run_query(gold_sql,  db_path)
    pred_res= run_query(pred_sql, db_path)
    ex_scores.append( int(gold_res == pred_res) )

# 5) aggregate & report
print("\n\nExact Match        :", np.mean(em_scores))
print("Component Match    :", np.mean(cm_scores))
print("Execution Accuracy :", np.mean(ex_scores))
print("Avg. Latency  (s)  :", np.mean(times))
print("95% Latency  (s)   :", np.percentile(times, 95))
if torch.cuda.is_available():
    print("GPU Mem Peak       :", torch.cuda.max_memory_allocated() / 1e9, "GB")
else:
    print("GPU Mem Peak       : N/A (CPU)")


Evaluating: 100%|██████████| 206/206 [25:16<00:00,  7.36s/it]



Exact Match        : 0.3106796116504854
Component Match    : 0.9069377237101724
Execution Accuracy : 0.6359223300970874
Avg. Latency  (s)  : 7.276892539024338
95% Latency  (s)   : 10.274793986000532
GPU Mem Peak       : 31.951907328 GB
